In [1]:
import cascade
from cascade import PyPoint
import pandas as pd
import geopandas as gpd
import osmnx as ox
import h3pandas

import time
import random
import string


def random_string(length=5):
    # Generate a random string of a given length (default: 5 characters)
    return "".join(random.choices(string.ascii_letters + string.digits, k=length))

In [2]:
gtfs_path = "/home/chingiz/Rust/py_rust/cascade/cascade-bin/files/SPB"
pbf_path = "/home/chingiz/Rust/osm/roads_SZ.pbf"
departure = 0
duration = 90000
weekday = "monday"

graph = cascade.create_graph(gtfs_path, pbf_path, departure, duration, weekday)

Filtering left 1942842 rows
Graph creation time: 17.092697499s


In [12]:
boundary = ox.geocode_to_gdf("Санкт-Петербург")
h3_grid = boundary.h3.polyfill(9, explode = True)
h3_grid.rename(columns = {"h3_polyfill": "index"}, inplace = True)
h3_grid.set_index("index", inplace = True)


h3_grid = h3_grid.h3.h3_to_geo_boundary()

In [13]:
h3_grid['id'] = h3_grid.index
hex_centroids = gpd.GeoDataFrame(h3_grid, geometry=h3_grid.geometry.centroid)

/tmp/ipykernel_65599/2080157066.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hex_centroids = gpd.GeoDataFrame(h3_grid, geometry=h3_grid.geometry.centroid)


In [14]:
def gdf_to_pypoint_list(gdf):
    points_list = []
    for _, row in gdf.iterrows():
        point_geom = row[
            "geometry"
        ]  # Assumes 'geometry' column has shapely Point objects
        py_point = PyPoint(x=point_geom.x, y=point_geom.y, id=str(row["id"]))
        points_list.append(py_point)
    return points_list

In [15]:
list_of_points = gdf_to_pypoint_list(hex_centroids)

In [7]:
start = time.perf_counter()
result = cascade.calculate_od_matrix(graph, list_of_points, 3600 * 12)
print("Time elapsed", time.perf_counter() - start, len(result))

# crate dataframe from result dict of dict
df_long: pd.DataFrame = (
    pd.DataFrame.from_dict(result, orient="index").fillna(0).stack().reset_index()
)
df_long.columns = ["from_point", "to_point", "weight"]

Time elapsed 52.488238876998366 963


In [8]:
df_long.head(2)

,from_point,to_point,weight
0,881106aa2bfffff,881106a9b1fffff,6277.840480
1,881106aa2bfffff,881106a9ddfffff,3970.478218


In [9]:
df2 = df_long.groupby("from_point")["weight"].mean().reset_index()
grid: gpd.GeoDataFrame = pd.merge(h3_grid, df2, left_on="id", right_on="from_point")

In [10]:
grid.explore(
    column="weight",
    cmap="RdYlGn_r",
    legend=True,
    tooltip="weight",
    legend_kwds=dict(colorbar=False),
    scheme="Quantiles",
    k=6,
)

In [11]:
(source_x, source_y)  = [(y,x) for (x,y) in [((59.864397, 30.304435))]][0]
(target_x, target_y) = [(y,x) for (x,y) in [(60.004281, 30.326125)]][0]

start = time.perf_counter()
print(
    cascade.shortest_path(
        graph=graph,
        dep_time=43200,
        source_x=source_x,
        source_y=source_y,
        target_x=target_x,
        target_y=target_y,
    )
)
print("Time elapsed", time.perf_counter() - start)

6173.840322292048
Time elapsed 0.322374569999738
